In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import requests
import geopandas as gpd
from pysal.lib import weights
from shapely import wkt
import pandas as pd
import arviz as az
import matplotlib.dates as mdates
import numpy as np
import causalpy as cp
import matplotlib.pyplot as plt
dr = DataReg()

In [ ]:
df_qcew = dr.base_data().with_columns(
    treatment=pl.when(pl.col("year") >= 2023).then(True).otherwise(False)
)
# df_dp03 = dr.pull_dp03()
pr_zips = gpd.GeoDataFrame(dr.make_spatial_table().df())
pr_zips["geometry"] = pr_zips["geometry"].apply(wkt.loads)
pr_zips = pr_zips.set_geometry("geometry")
pr_zips["zipcode"] = pr_zips["zipcode"].astype(str)

# df = df_qcew.join(df_dp03, on=["zipcode","year"], how="inner")
df = pr_zips.join(
    df_qcew.to_pandas().set_index("zipcode"), on="zipcode", how="inner", validate="1:m"
        ).reset_index(drop=True)
df = df.sort_values("year").reset_index(drop=True)
df

In [ ]:
sector_df = df[(df["sector"] == "61")].reset_index(drop=True)
y = sector_df["total_employment"].values.reshape(-1,1)
w = weights.distance.DistanceBand.from_dataframe(
    sector_df, 80467
)
reg = sector_df
reg["w_emplyment"] = weights.lag_spatial(w,y) 
reg

In [ ]:
# Assuming `reg` is your DataFrame and you want to set the 'year' column as a datetime index
data = reg.copy()
data = data.drop("geometry", axis=1)
data = data.drop("sector", axis=1)
data = data.drop("treatment", axis=1)
# Make sure the 'year' column is converted to datetime, assuming 'year' contains just the year (e.g., 2025)
data['year'] = pd.to_datetime(data['year'], format='%Y')

# Now set 'year' as the index and drop the 'year' column
data.index = data['year']
data = data.drop('year', axis=1)
data = data.drop("zipcode", axis=1)
data_grouped = data.groupby(data.index.year).mean()
data_grouped.index = pd.to_datetime(data_grouped.index, format='%Y')
data_grouped

In [ ]:
treatment_index = pd.to_datetime(2023, format='%Y')
model = cp.skl_models.WeightedProportion()
formula = 'total_employment ~ 0 + w_emplyment + k_index'

In [ ]:
results = cp.SyntheticControl(
data_grouped,
treatment_index,
formula=formula,
model=model,
)

In [ ]:
fig, ax = results.plot(plot_predictors=True)

In [ ]:
treatment_time = pd.to_datetime("2020-01-01")
result = cp.InterruptedTimeSeries(
    demeaned_data,
    treatment_time,
    formula="total_employment ~ 0 + k_index + total_employment",
    model=cp.pymc_models.LinearRegression(),
)